In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap
import joblib
from tensorflow.keras.models import load_model
from preload_explainers import preload_explainers, load_background_data, PredictProbaWrapper, EnsembleProbaWrapper
from utils import load_column_mapping, preprocess_single_transaction, load_model_from_disk
import os
import pickle
import streamlit as st
from IPython.display import display, HTML

# Import custom modules
from data_loader import (
    KerasBinaryClassifier, 
    load_transaction_data, 
    preprocess_data, 
    split_data, 
    train_random_forest
)
from model_explainer import (
    explain_prediction, 
    explain_model, 
    explain_with_lime, 
    plot_lime_explanation, 
    load_models_from_cache, 
    preprocess_single_transaction
)
from utils import load_column_mapping

# -----------------------------------------------------------------------------
# Now you can load your cached explainers without that AttributeError
# -----------------------------------------------------------------------------

# Load the column mapping
column_mapping = load_column_mapping()

# Set paths for model files
models_dir    = 'models'
rf_path       = os.path.join(models_dir, 'rf_model.joblib')
nn_path       = os.path.join(models_dir, 'nn_model.h5')
ensemble_path = os.path.join(models_dir, 'ensemble_model.joblib')
explainers_path = os.path.join(models_dir, 'explainers_cache.pkl')

# Load Random Forest model
print("Loading Random Forest model...")
rf_model = joblib.load(rf_path)

# Load Neural Network model
print("Loading Neural Network model...")
keras_model = load_model(nn_path)
nn_model    = KerasBinaryClassifier(model=keras_model,
                                    n_features=rf_model.n_features_in_)

# Load Ensemble model
print("Loading Ensemble model...")
ensemble_model = joblib.load(ensemble_path)

# Load explainers from cache
print("Loading explainers from cache...")
with open(explainers_path, 'rb') as f:
    explainers = pickle.load(f)

# Create dictionary of models
models = {
    'Random Forest':    rf_model,
    'Neural Network':   nn_model,
    'Ensemble':         ensemble_model
}

print("All models and explainers loaded successfully!")

# Create and preprocess sample transaction
print("\nPreparing sample transaction...")
feature_names = ['step', 'customer', 'age', 'gender', 'zipcodeOri', 
                 'merchant', 'zipMerchant', 'category', 'amount']

sample_transaction_dict = {
    'step': 1,
    'customer': 1,
    'age': 35,
    'gender': 1,
    'zipcodeOri': 0,
    'merchant': 1,
    'zipMerchant': 0,
    'category': 1,
    'amount': 100.0
}

processed_transaction = preprocess_single_transaction(sample_transaction_dict,
                                                     column_mapping)
processed_transaction = processed_transaction.reshape(1, -1)

print("Sample transaction preprocessed successfully!")
print(f"Processed transaction shape: {processed_transaction.shape}")


Loading Random Forest model...
Loading Neural Network model...
Loading Ensemble model...
Loading explainers from cache...
All models and explainers loaded successfully!

Preparing sample transaction...
Sample transaction preprocessed successfully!
Processed transaction shape: (1, 9)


In [8]:
# Test SHAP explanations for each model
print("\nGenerating SHAP explanations...")
for model_name, model in models.items():
    print(f"\nSHAP explanation for {model_name}:")
    
    # Get model prediction
    prediction = model.predict(processed_transaction)[0]
    prediction_prob = model.predict_proba(processed_transaction)[0]
    print(f"Model prediction: {'Fraud' if prediction == 1 else 'Not Fraud'}")
    print(f"Prediction probability: Fraud: {prediction_prob[1]:.3f}, Not Fraud: {prediction_prob[0]:.3f}")
    
    # Map model name to explainer key
    model_key = model_name.lower().replace(' ', '_')
    
    # Get SHAP values
    print("Force Plot")
    force_plot = explain_prediction(model_key, processed_transaction, feature_names, explainers)
    print("type of force_plot: ", type(force_plot))
    # force_plot.matplotlib(figsize=(10, 6), show=True, text_rotation=45)
    # plt.show()
    
    # Create SHAP waterfall plot
    print("Waterfall Plot")
    waterfall_plot = explain_prediction(model_key, processed_transaction, feature_names, explainers, plot_type='waterfall')
    print("type of waterfall_plot: ", type(waterfall_plot))
    # plt.show()
    
    # Create SHAP bar plot
    print("Bar Plot")
    bar_plot = explain_prediction(model_key, processed_transaction, feature_names, explainers, plot_type='bar')
    print("type of bar_plot: ", type(bar_plot))
    # plt.show()



Generating SHAP explanations...

SHAP explanation for Random Forest:
Model prediction: Not Fraud
Prediction probability: Fraud: 0.190, Not Fraud: 0.810
Force Plot
Transaction shape: (1, 9)
Model: random_forest
SHAP values shape: (1, 9, 2)
SHAP values: [[[ 0.06542628 -0.06542628]
  [ 0.10703478 -0.10703478]
  [ 0.01985286 -0.01985286]
  [-0.00444164  0.00444164]
  [ 0.          0.        ]
  [ 0.15876186 -0.15876186]
  [ 0.          0.        ]
  [ 0.17677422 -0.17677422]
  [-0.21358258  0.21358258]]]
Single output shape: (1, 9)
Single output: [[-0.06542628 -0.10703478 -0.01985286  0.00444164  0.         -0.15876186
   0.         -0.17677422  0.21358258]]
Expected value shape: (1, 2)
Expected value: [[0.50017421 0.49982579]]
type of force_plot:  <class 'shap.plots._force.AdditiveForceVisualizer'>
Waterfall Plot
Transaction shape: (1, 9)
Model: random_forest
SHAP values shape: (1, 9, 2)
SHAP values: [[[ 0.06542628 -0.06542628]
  [ 0.10703478 -0.10703478]
  [ 0.01985286 -0.01985286]
  [-

  0%|          | 0/1 [00:00<?, ?it/s]

Model: neural_network
SHAP values shape: (1, 9, 2)
SHAP values: [[[-3.33321396e-03  3.33321396e-03]
  [ 6.66678695e-03 -6.66678695e-03]
  [-1.28765901e-07  1.28765901e-07]
  [ 3.03685665e-08 -3.03685666e-08]
  [ 0.00000000e+00  0.00000000e+00]
  [-1.30494436e-07  1.30494436e-07]
  [ 0.00000000e+00  0.00000000e+00]
  [ 1.19725863e-07 -1.19725863e-07]
  [-3.33346383e-03  3.33346383e-03]]]
Single output shape: (1, 9)
Single output: [[ 3.33321396e-03 -6.66678695e-03  1.28765901e-07 -3.03685666e-08
   0.00000000e+00  1.30494436e-07  0.00000000e+00 -1.19725863e-07
   3.33346383e-03]]
Expected value shape: (1, 2)
Expected value: [[0. 1.]]
type of force_plot:  <class 'shap.plots._force.AdditiveForceVisualizer'>
Waterfall Plot
Transaction shape: (1, 9)


  0%|          | 0/1 [00:00<?, ?it/s]

Model: neural_network
SHAP values shape: (1, 9, 2)
SHAP values: [[[-3.33321396e-03  3.33321396e-03]
  [ 6.66678695e-03 -6.66678695e-03]
  [-1.28765901e-07  1.28765901e-07]
  [ 3.03685665e-08 -3.03685666e-08]
  [ 0.00000000e+00  0.00000000e+00]
  [-1.30494436e-07  1.30494436e-07]
  [ 0.00000000e+00  0.00000000e+00]
  [ 1.19725863e-07 -1.19725863e-07]
  [-3.33346383e-03  3.33346383e-03]]]
Single output shape: (1, 9)
Single output: [[ 3.33321396e-03 -6.66678695e-03  1.28765901e-07 -3.03685666e-08
   0.00000000e+00  1.30494436e-07  0.00000000e+00 -1.19725863e-07
   3.33346383e-03]]
Expected value shape: (1, 2)
Expected value: [[0. 1.]]
type of waterfall_plot:  <class 'matplotlib.axes._axes.Axes'>
Bar Plot
Transaction shape: (1, 9)


...

collecting probas:  [array([[0., 1.]], dtype=float32), array([[0.81, 0.19]])]
collecting probas:  [array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0.93, 0.07],
       [0.46, 0.54],
       [0.94, 0.06],
       ...,
       [0.94, 0.06],
       [0.94, 0.06],
       [0.94, 0.06]])]
Model: ensemble
SHAP values shape: (1, 9, 2)
SHAP values: [[[-0.0015      0.0015    ]
  [-0.00991667  0.00991667]
  [-0.00025     0.00025   ]
  [-0.013225    0.013225  ]
  [ 0.          0.        ]
  [-0.02031667  0.02031667]
  [ 0.          0.        ]
  [ 0.02680833 -0.02680833]
  [-0.0015      0.0015    ]]]
Single output shape: (1, 9)
Single output: [[ 0.0015      0.00991667  0.00025     0.013225    0.          0.02031667
   0.         -0.02680833  0.0015    ]]
Expected value shape: (1, 2)
Expected value: [[0.4249 0.5751]]
type of force_plot:  <class 'shap.plots._force.AdditiveForceVisualizer'>
Waterfall Plot
Transactio

  0%|          | 0/1 [00:00<?, ?it/s]

collecting probas:  [array([[0., 1.]], dtype=float32), array([[0.81, 0.19]])]
collecting probas:  [array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0.93, 0.07],
       [0.46, 0.54],
       [0.94, 0.06],
       ...,
       [0.94, 0.06],
       [0.94, 0.06],
       [0.94, 0.06]])]
Model: ensemble
SHAP values shape: (1, 9, 2)
SHAP values: [[[-0.0015      0.0015    ]
  [-0.00991667  0.00991667]
  [-0.00025     0.00025   ]
  [-0.013225    0.013225  ]
  [ 0.          0.        ]
  [-0.02031667  0.02031667]
  [ 0.          0.        ]
  [ 0.02680833 -0.02680833]
  [-0.0015      0.0015    ]]]
Single output shape: (1, 9)
Single output: [[ 0.0015      0.00991667  0.00025     0.013225    0.          0.02031667
   0.         -0.02680833  0.0015    ]]
Expected value shape: (1, 2)
Expected value: [[0.4249 0.5751]]
type of waterfall_plot:  <class 'matplotlib.axes._axes.Axes'>
Bar Plot
Transaction shape: (1, 9)


  0%|          | 0/1 [00:00<?, ?it/s]

collecting probas:  [array([[0., 1.]], dtype=float32), array([[0.81, 0.19]])]
collecting probas:  [array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32), array([[0.93, 0.07],
       [0.46, 0.54],
       [0.94, 0.06],
       ...,
       [0.94, 0.06],
       [0.94, 0.06],
       [0.94, 0.06]])]
Model: ensemble
SHAP values shape: (1, 9, 2)
SHAP values: [[[-0.0015      0.0015    ]
  [-0.00991667  0.00991667]
  [-0.00025     0.00025   ]
  [-0.013225    0.013225  ]
  [ 0.          0.        ]
  [-0.02031667  0.02031667]
  [ 0.          0.        ]
  [ 0.02680833 -0.02680833]
  [-0.0015      0.0015    ]]]
Single output shape: (1, 9)
Single output: [[ 0.0015      0.00991667  0.00025     0.013225    0.          0.02031667
   0.         -0.02680833  0.0015    ]]
Expected value shape: (1, 2)
Expected value: [[0.4249 0.5751]]
type of bar_plot:  <class 'matplotlib.axes._axes.Axes'>
